In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
d3 = pd.read_parquet(
    r"E:\Learning\TEAI Cup\Data\Parquet Data\d3_0_No_Cleaning.parquet"
)
d3

,qcpc_event_id,qcpc_event_machine_id,qcpc_event_machine_status_active_code,qcpc_event_create_timestamp,qcpc_event_machine_module_description,qcpc_event_part_number,qcpc_event_error_text,qcpc_event_machine_stop_code
0,27601227596,S-276,200,2025-12-08 23:33:41,Kamera,7-1452668-3,I-Kasten SP2: Cur= 1.421 LTL= 1.780 UTL= 1.880,0
1,27601226196,S-276,200,2025-12-05 11:30:08,Kamera,7-1452668-3,I-Kasten SP2: Cur= 1.895 LTL= 1.780 UTL= 1.880,0
2,26803927805,S-268,30000,2025-12-06 10:33:21,Bruderer,2-1703930-2,WERKZEUGSICHERUNG: KANAL7 STOERUNG,0
3,26903485075,S-269,30000,2025-12-08 21:48:34,Bruderer,2-1703930-1,WERKZEUGSICHERUNG: KANAL7 STOERUNG,0
4,22600234145,S-226,200,2025-12-08 07:13:26,Kamera,5-965906-1,Teil 1 Gap Y: Cur= 0.530 LTL= 0.470 UTL= 0.530,0
...,...,...,...,...,...,...,...,...
9910,27601228404,S-276,200,2025-12-09 04:49:32,Kamera,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 53.874 LTL= 0....,0
9911,26803918960,S-268,30000,2025-12-05 09:58:03,Bruderer,2-1703930-2,WERKZEUGSICHERUNG: KANAL7 STOERUNG,0
9912,26903485594,S-269,200,2025-12-09 13:37:47,Kamera,2-1703930-1,Spur 2 Iso: Cur= Nicht messbar LTL= 1.800 UTL...,0
9913,27601228836,S-276,200,2025-12-09 18:37:08,Kamera,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 51.532 LTL= 0....,0


In [3]:
# Maybe this data doesnt requires filtering just requires to be analysed

### Renaming to match other datasets

In [4]:
# Lets Rename to match other datasets
for i in d3.columns:
    lt = i.split("_")
    if len(lt) > 2:
        new_name = "_".join(lt[2:])
        d3.rename(columns={i: new_name}, inplace=True)
d3.rename(columns={"create_timestamp": "timestamp"}, inplace=True)
d3

,id,machine_id,machine_status_active_code,timestamp,machine_module_description,part_number,error_text,machine_stop_code
0,27601227596,S-276,200,2025-12-08 23:33:41,Kamera,7-1452668-3,I-Kasten SP2: Cur= 1.421 LTL= 1.780 UTL= 1.880,0
1,27601226196,S-276,200,2025-12-05 11:30:08,Kamera,7-1452668-3,I-Kasten SP2: Cur= 1.895 LTL= 1.780 UTL= 1.880,0
2,26803927805,S-268,30000,2025-12-06 10:33:21,Bruderer,2-1703930-2,WERKZEUGSICHERUNG: KANAL7 STOERUNG,0
3,26903485075,S-269,30000,2025-12-08 21:48:34,Bruderer,2-1703930-1,WERKZEUGSICHERUNG: KANAL7 STOERUNG,0
4,22600234145,S-226,200,2025-12-08 07:13:26,Kamera,5-965906-1,Teil 1 Gap Y: Cur= 0.530 LTL= 0.470 UTL= 0.530,0
...,...,...,...,...,...,...,...,...
9910,27601228404,S-276,200,2025-12-09 04:49:32,Kamera,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 53.874 LTL= 0....,0
9911,26803918960,S-268,30000,2025-12-05 09:58:03,Bruderer,2-1703930-2,WERKZEUGSICHERUNG: KANAL7 STOERUNG,0
9912,26903485594,S-269,200,2025-12-09 13:37:47,Kamera,2-1703930-1,Spur 2 Iso: Cur= Nicht messbar LTL= 1.800 UTL...,0
9913,27601228836,S-276,200,2025-12-09 18:37:08,Kamera,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 51.532 LTL= 0....,0


### Converting string time to actual timestamp

In [5]:
# Convert to datetime
d3["timestamp"] = pd.to_datetime(d3["timestamp"])


### Splitting Kamera and Bruderer and selecting Kamera

In [6]:
# Split Bruderer and Kamera

In [7]:
kamera_d3 = d3[d3.machine_module_description == "Kamera"]
kamera_d3

,id,machine_id,machine_status_active_code,timestamp,machine_module_description,part_number,error_text,machine_stop_code
0,27601227596,S-276,200,2025-12-08 23:33:41,Kamera,7-1452668-3,I-Kasten SP2: Cur= 1.421 LTL= 1.780 UTL= 1.880,0
1,27601226196,S-276,200,2025-12-05 11:30:08,Kamera,7-1452668-3,I-Kasten SP2: Cur= 1.895 LTL= 1.780 UTL= 1.880,0
4,22600234145,S-226,200,2025-12-08 07:13:26,Kamera,5-965906-1,Teil 1 Gap Y: Cur= 0.530 LTL= 0.470 UTL= 0.530,0
5,26903483904,S-269,200,2025-12-05 07:52:59,Kamera,2-1703930-1,Spur 2 Gap: Cur= Nicht messbar LTL= 0.160 UTL...,0
6,27601228449,S-276,200,2025-12-09 05:12:42,Kamera,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 45.405 LTL= 0....,0
...,...,...,...,...,...,...,...,...
9909,22300227531,S-223,200,2025-12-09 09:56:49,Kamera,5-963715-1,Teil 2 Gap Y: Cur= Nicht messbar LTL= 0.465 U...,0
9910,27601228404,S-276,200,2025-12-09 04:49:32,Kamera,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 53.874 LTL= 0....,0
9912,26903485594,S-269,200,2025-12-09 13:37:47,Kamera,2-1703930-1,Spur 2 Iso: Cur= Nicht messbar LTL= 1.800 UTL...,0
9913,27601228836,S-276,200,2025-12-09 18:37:08,Kamera,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 51.532 LTL= 0....,0


### Removing single valued columns

In [8]:
# Removing the parts with single value in them
columns_with_single_value = [col for col in kamera_d3.columns if kamera_d3[col].nunique() == 1]
print("columns_with_single_value:", columns_with_single_value)

# Removing these if they present
kamera_d3.drop(columns=columns_with_single_value, inplace=True)
kamera_d3

columns_with_single_value: ['machine_module_description', 'machine_stop_code']


,id,machine_id,machine_status_active_code,timestamp,part_number,error_text
0,27601227596,S-276,200,2025-12-08 23:33:41,7-1452668-3,I-Kasten SP2: Cur= 1.421 LTL= 1.780 UTL= 1.880
1,27601226196,S-276,200,2025-12-05 11:30:08,7-1452668-3,I-Kasten SP2: Cur= 1.895 LTL= 1.780 UTL= 1.880
4,22600234145,S-226,200,2025-12-08 07:13:26,5-965906-1,Teil 1 Gap Y: Cur= 0.530 LTL= 0.470 UTL= 0.530
5,26903483904,S-269,200,2025-12-05 07:52:59,2-1703930-1,Spur 2 Gap: Cur= Nicht messbar LTL= 0.160 UTL...
6,27601228449,S-276,200,2025-12-09 05:12:42,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 45.405 LTL= 0....
...,...,...,...,...,...,...
9909,22300227531,S-223,200,2025-12-09 09:56:49,5-963715-1,Teil 2 Gap Y: Cur= Nicht messbar LTL= 0.465 U...
9910,27601228404,S-276,200,2025-12-09 04:49:32,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 53.874 LTL= 0....
9912,26903485594,S-269,200,2025-12-09 13:37:47,2-1703930-1,Spur 2 Iso: Cur= Nicht messbar LTL= 1.800 UTL...
9913,27601228836,S-276,200,2025-12-09 18:37:08,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 51.532 LTL= 0....


## Analysing column -> error_text

In [9]:
# Analysis of error_test
sample = kamera_d3.error_text.sample()
sample

8963     Spur 2 I-Kasten mitte: Cur= Nicht messbar LTL...
Name: error_text, dtype: object

-  Lets spilt them and try if there are some values which does not contains curr, utl, ltl to check -> Split and check if last one is a float

In [10]:

def check(x):
    x = x.strip()
    parts = x.split(" ")
    if len(parts) < 2:
        return False
    try:
        float(parts[-1])
        return 0
    except ValueError:
        return 1



In [11]:
copy_kamersa_d3 = kamera_d3.copy()
copy_kamersa_d3["check"] = copy_kamersa_d3["error_text"].apply(check)
copy_kamersa_d3["check"].value_counts()

check
0    6487
Name: count, dtype: int64

In [12]:
# This shows all the values contains utl, ltl, and curr

### Splitting the values into variable, curr, ltl, utl

In [13]:
# Lets split and extract 3 values from error_text
def extract_values(x):
    out = pd.Series(x).str.extract(
        r"^(.*?):\s*Cur=\s*(.*?)\s+LTL=\s*(-?[0-9.]+)\s+UTL=\s*(-?[0-9.]+)"
    )
    out.columns = ["error_variable", "curr", "ltl", "utl"]
    
    return out.iloc[0]
    

In [14]:
kamera_d3[["error_variable", "curr", "ltl", "utl"]] = kamera_d3["error_text"].apply(extract_values)
kamera_d3

,id,machine_id,machine_status_active_code,timestamp,part_number,error_text,error_variable,curr,ltl,utl
0,27601227596,S-276,200,2025-12-08 23:33:41,7-1452668-3,I-Kasten SP2: Cur= 1.421 LTL= 1.780 UTL= 1.880,I-Kasten SP2,1.421,1.780,1.880
1,27601226196,S-276,200,2025-12-05 11:30:08,7-1452668-3,I-Kasten SP2: Cur= 1.895 LTL= 1.780 UTL= 1.880,I-Kasten SP2,1.895,1.780,1.880
4,22600234145,S-226,200,2025-12-08 07:13:26,5-965906-1,Teil 1 Gap Y: Cur= 0.530 LTL= 0.470 UTL= 0.530,Teil 1 Gap Y,0.530,0.470,0.530
5,26903483904,S-269,200,2025-12-05 07:52:59,2-1703930-1,Spur 2 Gap: Cur= Nicht messbar LTL= 0.160 UTL...,Spur 2 Gap,Nicht messbar,0.160,0.220
6,27601228449,S-276,200,2025-12-09 05:12:42,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 45.405 LTL= 0....,SP 1 durchgebrannt GW 70,45.405,0.000,45.000
...,...,...,...,...,...,...,...,...,...,...
9909,22300227531,S-223,200,2025-12-09 09:56:49,5-963715-1,Teil 2 Gap Y: Cur= Nicht messbar LTL= 0.465 U...,Teil 2 Gap Y,Nicht messbar,0.465,0.535
9910,27601228404,S-276,200,2025-12-09 04:49:32,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 53.874 LTL= 0....,SP 1 durchgebrannt GW 70,53.874,0.000,45.000
9912,26903485594,S-269,200,2025-12-09 13:37:47,2-1703930-1,Spur 2 Iso: Cur= Nicht messbar LTL= 1.800 UTL...,Spur 2 Iso,Nicht messbar,1.800,2.000
9913,27601228836,S-276,200,2025-12-09 18:37:08,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 51.532 LTL= 0....,SP 1 durchgebrannt GW 70,51.532,0.000,45.000


### Converting it to numeric

In [15]:
def convert_to_numeric(x):
    try:
        return pd.to_numeric(x)
    except ValueError:
        return np.nan

In [16]:
numeric_columns_to_convert = ["curr", "ltl", "utl"]
for col in numeric_columns_to_convert:
    kamera_d3[col] = kamera_d3[col].apply(convert_to_numeric)
kamera_d3

,id,machine_id,machine_status_active_code,timestamp,part_number,error_text,error_variable,curr,ltl,utl
0,27601227596,S-276,200,2025-12-08 23:33:41,7-1452668-3,I-Kasten SP2: Cur= 1.421 LTL= 1.780 UTL= 1.880,I-Kasten SP2,1.421,1.780,1.880
1,27601226196,S-276,200,2025-12-05 11:30:08,7-1452668-3,I-Kasten SP2: Cur= 1.895 LTL= 1.780 UTL= 1.880,I-Kasten SP2,1.895,1.780,1.880
4,22600234145,S-226,200,2025-12-08 07:13:26,5-965906-1,Teil 1 Gap Y: Cur= 0.530 LTL= 0.470 UTL= 0.530,Teil 1 Gap Y,0.530,0.470,0.530
5,26903483904,S-269,200,2025-12-05 07:52:59,2-1703930-1,Spur 2 Gap: Cur= Nicht messbar LTL= 0.160 UTL...,Spur 2 Gap,NaN,0.160,0.220
6,27601228449,S-276,200,2025-12-09 05:12:42,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 45.405 LTL= 0....,SP 1 durchgebrannt GW 70,45.405,0.000,45.000
...,...,...,...,...,...,...,...,...,...,...
9909,22300227531,S-223,200,2025-12-09 09:56:49,5-963715-1,Teil 2 Gap Y: Cur= Nicht messbar LTL= 0.465 U...,Teil 2 Gap Y,NaN,0.465,0.535
9910,27601228404,S-276,200,2025-12-09 04:49:32,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 53.874 LTL= 0....,SP 1 durchgebrannt GW 70,53.874,0.000,45.000
9912,26903485594,S-269,200,2025-12-09 13:37:47,2-1703930-1,Spur 2 Iso: Cur= Nicht messbar LTL= 1.800 UTL...,Spur 2 Iso,NaN,1.800,2.000
9913,27601228836,S-276,200,2025-12-09 18:37:08,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 51.532 LTL= 0....,SP 1 durchgebrannt GW 70,51.532,0.000,45.000


- Removing single values columns after converting

In [17]:
columns_to_remove = ["error_text", 'id']
for i in columns_to_remove:
    if i in kamera_d3.columns:
        kamera_d3.drop(columns=[i], inplace=True)
kamera_d3

,machine_id,machine_status_active_code,timestamp,part_number,error_variable,curr,ltl,utl
0,S-276,200,2025-12-08 23:33:41,7-1452668-3,I-Kasten SP2,1.421,1.780,1.880
1,S-276,200,2025-12-05 11:30:08,7-1452668-3,I-Kasten SP2,1.895,1.780,1.880
4,S-226,200,2025-12-08 07:13:26,5-965906-1,Teil 1 Gap Y,0.530,0.470,0.530
5,S-269,200,2025-12-05 07:52:59,2-1703930-1,Spur 2 Gap,NaN,0.160,0.220
6,S-276,200,2025-12-09 05:12:42,7-1452668-3,SP 1 durchgebrannt GW 70,45.405,0.000,45.000
...,...,...,...,...,...,...,...,...
9909,S-223,200,2025-12-09 09:56:49,5-963715-1,Teil 2 Gap Y,NaN,0.465,0.535
9910,S-276,200,2025-12-09 04:49:32,7-1452668-3,SP 1 durchgebrannt GW 70,53.874,0.000,45.000
9912,S-269,200,2025-12-09 13:37:47,2-1703930-1,Spur 2 Iso,NaN,1.800,2.000
9913,S-276,200,2025-12-09 18:37:08,7-1452668-3,SP 1 durchgebrannt GW 70,51.532,0.000,45.000


- Sorting based on timestamp

In [18]:
kamera_d3.sort_values(by=["timestamp"], inplace=True)
kamera_d3.reset_index(inplace=True, drop=True)
kamera_d3

,machine_id,machine_status_active_code,timestamp,part_number,error_variable,curr,ltl,utl
0,S-276,200,2025-12-05 00:00:18,7-1452668-3,GAP rechts,0.280,0.360,0.420
1,S-269,200,2025-12-05 00:07:25,2-1703930-1,Spur 1 Gap,0.222,0.160,0.220
2,S-276,200,2025-12-05 00:08:58,7-1452668-3,I-Freischnitt Lamelle unten,92.000,25.000,75.000
3,S-276,200,2025-12-05 00:08:58,7-1452668-3,GAP rechts,NaN,0.360,0.420
4,S-276,200,2025-12-05 00:08:58,7-1452668-3,Gap links,0.443,0.360,0.420
...,...,...,...,...,...,...,...,...
6482,S-276,200,2025-12-09 20:59:37,7-1452668-3,SP 1 durchgebrannt GW 70,52.973,0.000,45.000
6483,S-223,200,2025-12-09 21:00:19,5-963715-1,Teil 2 Bodylötstelle,0.000,0.900,2.000
6484,S-223,200,2025-12-09 21:00:19,5-963715-1,Teil 2 Bodylötstelle,0.000,0.900,2.000
6485,S-223,200,2025-12-09 21:00:19,5-963715-1,Teil 2 Gap Y,0.464,0.465,0.535


## Saving the cleaned dataset

In [19]:
kamera_d3.to_parquet(r"E:\Learning\TEAI Cup\Data\Parquet Data\d3_1_General_Cleaning.parquet")